In [2]:

import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import re

df = pd.read_csv('daily_update.csv', index_col=0, parse_dates=True, names = ['date', 'vaccine_num_total', 'vaccine_num_daily', 'link'])
df = df.iloc[1:]
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['vaccine_num_total'] = df['vaccine_num_total'].astype(int)
df['vaccine_num_daily'] = df['vaccine_num_daily'].astype(int)

latest_date_org = df['date'].max()
latest_cavvine_num = df[df['date'] == latest_date_org]['vaccine_num_total'].values[0]

# Get the official link of the latest vaccine data
url = 'http://sousuo.gov.cn/column/40403/0.htm'
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

links = soup.find_all('li')
max_date = datetime.date(2020, 12, 1)
max_date_link = ''
for i in links:
    if '新冠病毒疫苗接种情况' in i.text:
        vaccine_announce_date = i.a['href'][25:35]
        vaccine_announce_date = datetime.datetime.strptime(vaccine_announce_date, '%Y-%m/%d').date()
        vaccine_announce_date = vaccine_announce_date + datetime.timedelta(days=-1)

        if vaccine_announce_date > max_date:
            max_date = vaccine_announce_date
            link = i.a['href']
            max_date_link = link
print(max_date)
print(max_date_link)


# Get the latest vaccine data
if max_date > latest_date_org:
    url = max_date_link
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    soup
    page_text =  soup.find('p').text
    text_extract = page_text[20:]
    text_num = re.findall(r'(\d+\.\d+)', text_extract)
    text_num = text_num[0]
    vaccine_num = int(float(text_num)*10000)
    print(vaccine_num)
    text_date = re.findall(r'(\d{4}年\d{1,2}月\d{1,2}日)', page_text)[0]
    text_date = datetime.datetime.strptime(text_date, '%Y年%m月%d日').date()
    print(text_date)
    vaccine_num_daily = vaccine_num - latest_cavvine_num
    print(vaccine_num_daily)
    print(page_text)
    df = df.append({'date': text_date, 'vaccine_num_total': vaccine_num, 'vaccine_num_daily': vaccine_num_daily, 'link': url}, ignore_index=True)
    df.to_csv('daily_update.csv')
    print('New data added')
else:
    print('No new data')

2022-12-14
http://www.gov.cn/xinwen/2022-12/15/content_5732194.htm
3453156000
2022-12-14
1479000
截至2022年12月14日，31个省（自治区、直辖市）和新疆生产建设兵团累计报告接种新冠病毒疫苗345315.6万剂次。
New data added


/var/folders/j_/_3yzbdjs3ylf4k27zrqrssqc0000gp/T/ipykernel_66431/3497164240.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': text_date, 'vaccine_num_total': vaccine_num, 'vaccine_num_daily': vaccine_num_daily, 'link': url}, ignore_index=True)


In [3]:
#getting the data in the past 90 days
df = pd.read_csv('daily_update.csv', index_col=0, parse_dates=True, names = ['date', 'vaccine_num_total', 'vaccine_num_daily', 'link'])
df = df.iloc[1:]
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['vaccine_num_total'] = df['vaccine_num_total'].astype(int)
df['vaccine_num_daily'] = df['vaccine_num_daily'].astype(int)

latest_date_org = df['date'].max()
date_minus_180 = latest_date_org + datetime.timedelta(days=-180)
df = df[df['date'] >= date_minus_180]

df.to_csv('daily_update_180days.csv')
